In [ ]:
!pip install imutils

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam,Adamax
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import shutil
import random
import cv2
import argparse
import os
import pandas as pd

In [ ]:
!gdown --id 1bFrktIYGOp9cU7o6QBL8cv2QGToONh-W
!gdown --id 1uCB5XydEqICQnGCWRWc2xXCg0exLhjc_

In [ ]:
import pandas as pd
import numpy as np 
import os 
import shutil
import glob
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
! conda install openpyxl -y

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

plt.rcParams['figure.figsize'] = [8.0, 8.0]
plt.rcParams['figure.dpi'] = 100

In [ ]:
datanom = pd.read_excel('../input/normal/Normal.metadata.xlsx')
datanom['FILE NAME'][0]
datacov = pd.read_excel('../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID.metadata.xlsx')
datacov['FILE NAME'][0]

In [ ]:
import os
output = '/kaggle/working/data/'
try:
    os.mkdir(output)
    os.mkdir(output+'/Normal')
    os.mkdir(output+'/COVID')

except Exception as e :
    print(e)

In [ ]:
for i in range(1500): #1500
    dest = './data/Normal/'+datanom['FILE NAME'][i]+'.png'
    src = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/'+datanom['FILE NAME'][i]+'.png' #//content/gdrive/My Drive/data_set/all/
    shutil.copyfile(src, dest)
    dest = './data/COVID/'+datacov['FILE NAME'][i]+'.png'
    src = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/'+datacov['FILE NAME'][i]+'.png' #//content/gdrive/My Drive/data_set/all/
    shutil.copyfile(src, dest)
print('ok')

In [ ]:
dataset_path='./data'
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset_path))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)
print("[INFO] Images successfully loaded")

In [ ]:
covid_imgs = pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID.metadata.xlsx", engine='openpyxl')
covid_imgs.head(2)

In [ ]:
opacity_imgs = pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity.metadata.xlsx", engine='openpyxl')
opacity_imgs.head(2)

In [ ]:
normal_imgs = pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal.metadata.xlsx", engine='openpyxl')
normal_imgs.head(2)

In [ ]:
pneumonia_imgs = pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia.metadata.xlsx", engine='openpyxl')
pneumonia_imgs.head(2)

In [ ]:
ROOT_DIR = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/"
imgs = ['COVID','Lung_Opacity','Normal','Viral Pneumonia']

NEW_DIR = "./all_images/"

In [ ]:
print(imgs[0])
print(imgs[1])
print(imgs[2])
print(imgs[3])

In [ ]:
if not os.path.exists(NEW_DIR):
    os.mkdir(NEW_DIR)
    
    for i in imgs:
        org_dir = os.path.join(ROOT_DIR,i+"/")
        
        for imgfile in glob.iglob(os.path.join(org_dir,"*.png")):
            shutil.copy(imgfile,NEW_DIR)
else:
    print("Already Exist")

In [ ]:
counter = {"COVID":0,"Lung_Opacity":0,"Normal":0,"Viral Pneumonia":0}

for image in imgs:
    for count in glob.iglob(NEW_DIR+image+"*"):
        counter[image] += 1

print(counter)

In [ ]:
plt.rcParams['figure.figsize'] = [15,7]
plt.bar(x=counter.keys(),height=counter.values())
plt.show()

In [ ]:
if not os.path.exists(NEW_DIR+"train_test_split/"):
    
    os.makedirs(NEW_DIR+"train_test_split/")
    
    os.makedirs(NEW_DIR+"train_test_split/train/COVID")
    os.makedirs(NEW_DIR+"train_test_split/test/COVID")
    os.makedirs(NEW_DIR+"train_test_split/validation/COVID")
    
    os.makedirs(NEW_DIR+"train_test_split/train/Normal")
    os.makedirs(NEW_DIR+"train_test_split/test/Normal")
    os.makedirs(NEW_DIR+"train_test_split/validation/Normal")
    
    os.makedirs(NEW_DIR+"train_test_split/train/Lung Opacity")
    os.makedirs(NEW_DIR+"train_test_split/test/Lung Opacity")
    os.makedirs(NEW_DIR+"train_test_split/validation/Lung Opacity")
    
    os.makedirs(NEW_DIR+"train_test_split/train/Viral Pneumonia")
    os.makedirs(NEW_DIR+"train_test_split/test/Viral Pneumonia")
    os.makedirs(NEW_DIR+"train_test_split/validation/Viral Pneumonia")
    
    for i in np.random.choice(replace= False , size= 2531 , a = glob.glob(NEW_DIR+imgs[0]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/train/Covid" )
        os.remove(i)

    for i in np.random.choice(replace= False , size= 7134 , a = glob.glob(NEW_DIR+imgs[2]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/train/Normal" )
        os.remove(i)
        
    for i in np.random.choice(replace= False , size= 4208 , a = glob.glob(NEW_DIR+imgs[1]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/train/Lung Opacity" )
        os.remove(i)

    for i in np.random.choice(replace= False , size= 941 , a = glob.glob(NEW_DIR+imgs[3]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/train/Viral Pneumonia" )
        os.remove(i)

    # Validation Data
    for i in np.random.choice(replace= False , size= 723 , a = glob.glob(NEW_DIR+imgs[0]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/validation/Covid" )
        os.remove(i)

    for i in np.random.choice(replace= False , size= 2038 , a = glob.glob(NEW_DIR+imgs[2]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/validation/Normal" )
        os.remove(i)
        
    for i in np.random.choice(replace= False , size= 1202 , a = glob.glob(NEW_DIR+imgs[1]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/validation/Lung Opacity" )
        os.remove(i)

    for i in np.random.choice(replace= False , size= 269 , a = glob.glob(NEW_DIR+imgs[3]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/validation/Viral Pneumonia" )
        os.remove(i)


    # Test Data
    for i in np.random.choice(replace= False , size= 362 , a = glob.glob(NEW_DIR+imgs[0]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/test/Covid" )
        os.remove(i)

    for i in np.random.choice(replace= False , size= 1020 , a = glob.glob(NEW_DIR+imgs[2]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/test/Normal" )
        os.remove(i)
        
    for i in np.random.choice(replace= False , size= 602 , a = glob.glob(NEW_DIR+imgs[1]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/test/Lung Opacity" )
        os.remove(i)

    for i in np.random.choice(replace= False , size= 135 , a = glob.glob(NEW_DIR+imgs[3]+"*") ):
        shutil.copy(i , NEW_DIR+"train_test_split/test/Viral Pneumonia" )
        os.remove(i)


In [ ]:
train_path  = "./all_images/train_test_split/train"
valid_path  = "./all_images/train_test_split/validation"
test_path   = "./all_images/train_test_split/test"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from keras.models import Model
from keras.layers import Dense, MaxPool2D, Conv2D
import keras

In [ ]:
train_data_gen = ImageDataGenerator(preprocessing_function= vgg16.preprocess_input , zoom_range= 0.2, horizontal_flip= True, shear_range= 0.2 , rescale= 1./255)
train = train_data_gen.flow_from_directory(directory= train_path , target_size=(224,224))

In [ ]:
validation_data_gen = ImageDataGenerator(preprocessing_function= vgg16.preprocess_input , rescale= 1./255 )
valid = validation_data_gen.flow_from_directory(directory= valid_path , target_size=(224,224))

In [ ]:
test_data_gen = ImageDataGenerator(preprocessing_function= vgg16.preprocess_input, rescale= 1./255 )
test = train_data_gen.flow_from_directory(directory= test_path , target_size=(224,224), shuffle= False)

In [ ]:
train.class_indices

In [ ]:
class_type = {0:'COVID', 1:'Lung Opacity', 2:'Normal', 3:'Viral Pneumonia'}

In [ ]:
t_img , label = train.next()

In [ ]:
len(t_img)

In [ ]:
def plotImages(img_arr, label):
    
  for im, l in zip(img_arr,label) :
    plt.figure(figsize= (5,5))
    plt.imshow(im, cmap = 'gray')
    plt.title(im.shape)
    plt.axis = False
    plt.show()

In [ ]:
plotImages(t_img, label)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D,AveragePooling2D
from tensorflow.keras.optimizers import Adam,Adamax
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.xception import Xception
from tensorflow import keras
import matplotlib.cm as cm
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time


In [ ]:
# Now we will create a One-Hot encoding to the list of labels to make the classfication
# integer encode
lb_encoder = LabelEncoder()
labels = lb_encoder.fit_transform(labels)
labels = to_categorical(labels)


# Split the data into training and testing using the 80% of training and 20% to testing
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, stratify = labels,random_state = 42)


# Set the image augmentation of the training data
trainAug = ImageDataGenerator(rotation_range= 15, fill_mode='nearest')

In [ ]:
INIT_LR = 1e-3
EPOCHS = 100
BS = 8
base_model = Xception(weights = 'imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
headmodel = base_model.output
headmodel = AveragePooling2D(pool_size =(4, 4))(headmodel)
headmodel = Flatten(name ='Flatten')(headmodel)
headmodel = Dense(64, activation = 'relu')(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(4, activation = 'softmax')(headmodel)#2>Dense(2, activation = 'softmax')(headmodel)

model = Model(inputs = base_model.input, outputs = headmodel)

for layers in base_model.layers:
    layers.trainable = False


opt = Adam(lr = INIT_LR, decay = INIT_LR/EPOCHS)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])


In [ ]:
model.summary()

In [ ]:
with tf.device('/gpu:0'):
    print("Training the model with gpu . . .")
#     hist = model.fit_generator(trainAug.flow(X_train, y_train, batch_size = BS),steps_per_epoch=len(X_train) // BS, validation_data=(X_test, y_test), validation_steps=len(X_test) // BS, epochs=100)
    hist = model.fit_generator(train, steps_per_epoch= len(train) // BS, epochs= 100, validation_data= valid , validation_steps= len(test) // BS)#, callbacks=[es,mc])

In [ ]:
model.save("xception.h5")

In [ ]:
plt.subplot(211)
plt.title('Categorical Crossentropy Loss')
plt.plot(hist.history['loss'], color ='red', label='train')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.plot(hist.history['val_loss'], color ='green', label='val')

plt.show()

plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(hist.history['accuracy'], color='red', label='train')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.plot(hist.history['val_accuracy'], color='green', label='test')

plt.show()

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(X_test, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(y_test.argmax(axis=1), predIdxs, target_names=lb_encoder.classes_))

In [ ]:
# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(y_test.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
# show the accuracy, sensitivity, and specificity of the test
print("accuracy: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

In [ ]:
model.save('./vgg19.h5')
import pandas as pd
hist_df = pd.DataFrame(hist.history) 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                           normalize=False,
                           title='Confusion matrix',
                           cmap=plt.cm.Blues):
     """
     This function prints and plots the confusion matrix.
     Normalization can be applied by setting `normalize=True`.
     """
     
     plt.imshow(cm, interpolation='nearest', cmap=cmap)
     plt.title(title)
     plt.colorbar()
     tick_marks = np.arange(len(classes))
     plt.xticks(tick_marks, classes, rotation=45)
     plt.yticks(tick_marks, classes)

     if normalize:
         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
         print("Normalized confusion matrix")
     else:
         print('Confusion matrix, without normalization')

     print(cm)

     thresh = cm.max() / 2.
     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
         plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

     plt.tight_layout()
     plt.ylabel('True label')
     plt.xlabel('Predicted label')
     plt.show()

In [ ]:
def score(model,shape):
  predict = model.predict(X_test, batch_size=BS)
  predict = np.argmax(predict, axis=1)
  cm = confusion_matrix(y_test.argmax(axis=1), predict)
  cm_plot_labels = ['COVID-19','Normal']
  plot_confusion_matrix(cm, classes=cm_plot_labels, title='Confusion Matrix') 
  accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
  recall = (cm[0][0])/(cm[0][0]+cm[1][0])
  presition = (cm[0][0])/(cm[0][0]+cm[0][1])
  specificity = (cm[1][1])/(cm[1][1]+cm[0][1])
  f1 = 2*presition*recall/(presition+recall)
  print(f'accuracy = {accuracy}\nrecall = {recall}\npresition = {presition}\nspecificity = {specificity}\nf1 = {f1}')
  fpr, tpr, threshold = metrics.roc_curve(y_test.argmax(axis=1), predict)
  roc_auc = metrics.auc(fpr, tpr)
  plt.title('Receiver Operating Characteristic')
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()

In [ ]:
import sklearn.metrics as metrics

score(model,224)

In [ ]:
def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size "size"
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Rescale heatmap to a range 0-255
    img = img*255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap*0.4+img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
#     display(Image(cam_path))
    
    return superimposed_img
    
preprocess_input = tf.keras.applications.vgg19.preprocess_input
decode_predictions = tf.keras.applications.vgg19.decode_predictions

last_conv_layer_name = "block5_conv3"
img_size = (224,224)

# Remove last layer's softmax
model.layers[-1].activation = None

In [ ]:
img_path = '/content/gdrive/MyDrive/data_set/test/COVID/COVID-58.png'
BS=8
from matplotlib import cm
plt.figure(figsize=(12, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    i=i+100
    array = np.expand_dims(X_test[i], axis=0)
    heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name)
    cam_path = save_and_display_gradcam(X_test[i], heatmap)
    plt.imshow(cam_path)
    predict = model.predict(X_test, batch_size=BS)
    predict = np.argmax(predict, axis=1)
    plt.title(f'{lb_encoder.inverse_transform(y_test.argmax(axis=1))[i]}==>{lb_encoder.inverse_transform(predict)[i]} ')
plt.tight_layout()
plt.show()



In [ ]:
model.save('./vgg19.h5')